In [1]:
df = spark.read.csv('../data/sales.csv', header=True, inferSchema=True)

In [2]:
df.show(5)

+----------------+--------+-----+------------+-----------------+--------------------+--------+--------------+----------------+----------------+-----------+----------+
|Transaction_date| Product|Price|Payment_Type|             Name|                City|   State|       Country| Account_Created|      Last_Login|   Latitude| Longitude|
+----------------+--------+-----+------------+-----------------+--------------------+--------+--------------+----------------+----------------+-----------+----------+
| 01/02/2009 6:17|Product1| 1200|  Mastercard|         carolina|            Basildon| England|United Kingdom| 01/02/2009 6:00| 01/02/2009 6:08|       51.5|-1.1166667|
| 01/02/2009 4:53|Product1| 1200|        Visa|           Betina|Parkville        ...|      MO| United States| 01/02/2009 4:42| 01/02/2009 7:49|     39.195| -94.68194|
|01/02/2009 13:08|Product1| 1200|  Mastercard|Federica e Andrea|Astoria          ...|      OR| United States|01/01/2009 16:21|01/03/2009 12:32|   46.18806|   -123.83

In [5]:
df.select('Country').distinct().show()

+-------------+
|      Country|
+-------------+
|       Sweden|
|       Jersey|
|     Malaysia|
|       Turkey|
|      Germany|
|       France|
|      Belgium|
|      Finland|
|United States|
|        India|
|       Kuwait|
|        Malta|
|        Italy|
|       Norway|
|        Spain|
|      Denmark|
|      Ireland|
|       Israel|
|      Iceland|
|  South Korea|
+-------------+
only showing top 20 rows



In [15]:
# df.where("Country == 'Brazil'").select('Name').show()


+-------+
|   Name|
+-------+
|Joachim|
|  Diana|
+-------+



In [93]:
df.groupBy("Country").sum("Price").show()

+-------------+----------+
|      Country|sum(Price)|
+-------------+----------+
|       Sweden|      8400|
|       Jersey|      1200|
|     Malaysia|      1200|
|       Turkey|      2400|
|      Germany|     22800|
|       France|     30300|
|      Belgium|      3600|
|      Finland|      1200|
|United States|    350350|
|        India|      2400|
|       Kuwait|      1200|
|        Malta|      3600|
|        Italy|      2400|
|       Norway|     12000|
|        Spain|      2400|
|      Denmark|      8400|
|      Ireland|     29100|
|       Israel|      1200|
|      Iceland|      1200|
|  South Korea|      1200|
+-------------+----------+
only showing top 20 rows



In [46]:
df.groupBy("Country").sum("Price").orderBy("sum(Price)", ascending = False).show()

+--------------------+----------+
|             Country|sum(Price)|
+--------------------+----------+
|       United States|    350350|
|      United Kingdom|     63600|
|              Canada|     42000|
|              France|     30300|
|             Ireland|     29100|
|           Australia|     22800|
|             Germany|     22800|
|         Switzerland|     19200|
|         Netherlands|     14400|
|              Norway|     12000|
|              Brazil|      8700|
|              Sweden|      8400|
|             Denmark|      8400|
|               Malta|      3600|
|United Arab Emirates|      3600|
|             Austria|      3600|
|             Belgium|      3600|
|        South Africa|      3600|
|              Turkey|      2400|
|               India|      2400|
+--------------------+----------+
only showing top 20 rows



In [50]:
df2 = spark.read.csv('../data/countries.csv', header=True, inferSchema=True)

In [79]:
df.join(df2,"Country").select("ID", "Price").groupBy("ID").sum("Price").orderBy("sum(Price)", ascending = False).show()

+---+----------+
| ID|sum(Price)|
+---+----------+
|  2|    350350|
|  1|     63600|
|  8|     42000|
|  5|     30300|
|  7|     29100|
| 19|     22800|
|  3|     22800|
| 12|     19200|
|  6|     14400|
| 16|     12000|
| 33|      8700|
| 13|      8400|
| 15|      8400|
| 26|      3600|
| 22|      3600|
| 10|      3600|
| 28|      3600|
| 14|      3600|
| 34|      2400|
|  9|      2400|
+---+----------+
only showing top 20 rows



In [78]:
df.groupBy("Name").count().count()

378

In [6]:
from pyspark.sql.functions import *

numOfIterations = 10

lines = spark.read.text("../data/pagerank_data.txt")
# You can also test your program on the follow larger data set:
# lines = spark.read.text("../data/dblp.in")

a = lines.select(split(lines[0],' '))
# print(a.show())
links = a.select(a[0][0].alias('src'), a[0][1].alias('dst'))
# print(links.show())
outdegrees = links.groupBy('src').count()
# print(outdegrees.show())
ranks = outdegrees.select('src', lit(1).alias('rank'))
# tmp1 = ranks.withColumnRenamed('src', 'dst').join(links, 'dst').groupBy('src').count()  
# tmp2 = links.join(ranks, 'src')
# tmp3 = tmp2.join(tmp1, 'src')
# tmp3 = tmp3.withColumn('new', (col('rank')/(col('count'))))
# tmp3 = tmp3.select("dst",'new').groupBy('dst').agg(sum("new").alias('rank'))
# tmp3 = tmp3.withColumn('rank', col('rank') * 0.85 + 0.15).withColumnRenamed('dst', 'src')

# tmp3.withColumn("newRank",\
#     (col('rank')/col('count')))\
#     .groupBy('dst')\
#     .sum('newRank')\
#     .withColumnRenamed('sum(newRank)', 'rank')\
#     .withColumnRenamed('dst', 'src')\
#     .withColumn('rank', col('rank') * 0.85 + 0.15)

for iteration in range(numOfIterations):
# # # # FILL IN THIS PART
    tmp1 = ranks.select(col('src').alias('dst')).join(links, 'dst').groupBy('src').count().persist()
    tmp2 = links.join(ranks, 'src').persist()
    tmp3 = tmp2.join(tmp1, 'src')
    tmp3 = tmp3.select('dst',((col("rank")/col("count"))).alias("new"))
    tmp3 = tmp3.select("dst",'new').groupBy('dst').agg(sum("new").alias('rank'))
    ranks = tmp3.select(col('dst').alias('src'), (tmp3.rank*0.85 + 0.15).alias('rank'))
    if iteration > 7:
        ranks.cache()
        ranks.show()

ranks.orderBy(desc('rank')).show()

+---+------------------+
|src|              rank|
+---+------------------+
|  3|0.9999999999999998|
|  1|1.2983805334459568|
|  4|0.9999999999999998|
|  2|0.7016194665540426|
+---+------------------+



Py4JJavaError: An error occurred while calling o1931.showString.
: java.lang.OutOfMemoryError: Java heap space
	at java.base/java.lang.AbstractStringBuilder.<init>(AbstractStringBuilder.java:86)
	at java.base/java.lang.StringBuilder.<init>(StringBuilder.java:116)
	at org.apache.spark.sql.catalyst.util.StringUtils$StringConcat.toString(StringUtils.scala:147)
	at org.apache.spark.sql.catalyst.util.StringUtils$PlanStringConcat.toString(StringUtils.scala:174)
	at org.apache.spark.sql.execution.QueryExecution.explainString(QueryExecution.scala:220)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:103)
	at org.apache.spark.sql.execution.SQLExecution$$$Lambda$2275/0x0000000840f9d040.apply(Unknown Source)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:169)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$1(SQLExecution.scala:95)
	at org.apache.spark.sql.execution.SQLExecution